<a href="https://colab.research.google.com/github/batust/Pre-processing/blob/main/pre-prcoessor-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install presidio_analyzer
!pip install presidio_anonymizer
!python -m spacy download en_core_web_lg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [37]:
from presidio_analyzer import AnalyzerEngine, AnonymizerEngine


text = """"He is Allwyn Bat Thomas, having 2 emails -> allwynbatthomas@ust.com and b20cs100@mace.ac.in. Khan's phone number is +973 30982167
I'm based on Wakanda, AFrica."""

json_data = {
    "name": "Allwyn Bat Thomas",
    "email": "allwynbatthomas@ust.com",
    "backup_email": "b20cs100@mace.ac.in",
    "phone": "+973 30982167",
    "location": "Wakanda, Africa",
    "description": "Allwyn is a software engineer from Wakanda."
}

analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

def json_data(data):


results = analyzer.analyze(text=text, entities=["PHONE_NUMBER", "EMAIL_ADDRESS", "PERSON", "LOCATION"], language="en")

anonymizer = AnonymizerEngine()
anonymized_text = anonymizer.anonymize(text=text, analyzer_results=results)

print(anonymized_text.text)



"He is <PERSON>, having 2 emails -> <EMAIL_ADDRESS> and <EMAIL_ADDRESS>. <PERSON>'s phone number is <PHONE_NUMBER>
I'm based on <LOCATION>, AFrica.


In [38]:
from presidio_analyzer import AnalyzerEngine

text = 'His name is John Cena from Maryland, USA'
analyzer = AnalyzerEngine()
results = analyzer.analyze(text=text, language='en')

print(results)


[type: PERSON, start: 12, end: 21, score: 0.85, type: LOCATION, start: 27, end: 35, score: 0.85, type: LOCATION, start: 37, end: 40, score: 0.85]


In [40]:
from presidio_analyzer import PatternRecognizer, AnalyzerEngine

titles_list = [
    "Sir",
    "Ma'am",
    "Madam",
    "Mr.",
    "Mrs.",
    "Ms.",
    "Miss",
    "Dr.",
    "Professor",
]

text = 'Sir Allwyn Bat Thomas is a CIA agent!'

pattern = PatternRecognizer(
    supported_entity="TITLE",
    deny_list=titles_list,
)

result = pattern.analyze(
    text=text,
    entities=['TITLE']
)
print(f'Result: {result}')

analyzer = AnalyzerEngine()
analyzer.registry.add_recognizer(pattern)
ans = analyzer.analyze(text=text, language='en')
print(ans)

Result: [type: TITLE, start: 0, end: 3, score: 1.0]


[type: TITLE, start: 0, end: 3, score: 1.0, type: PERSON, start: 4, end: 21, score: 0.85]


In [1]:
json_data = {
    "name": "Allwyn Bat Thomas",
    "email":[ "allwynbatthomas@ust.com", 'allwynt700@gmail.com', '287945@ust.com'],
    "backup_email": "b20cs100@mace.ac.in",
    "phone": "+973 30982167",
    "location": "Wakanda, Africa",
    "description": "Allwyn is a software engineer from Wakanda.",
    "metadata": {
        "emergency_contact": "+91 9678785654",
        "company": "UST Global",
        "address": {
            "city": "New York",
            "country": "USA"
        }
    }
}

In [23]:
# Uploading json file and doing NER on attributes
import json
from google.colab import files
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

def processing_json(data):
  if isinstance(data, dict):
    return {key: processing_json(value) for key, value in data.items()}
  elif isinstance(data, list):
    return [processing_json(item) for item in data]
  elif isinstance(data, str):
    result = analyzer.analyze(
        text = data,
        entities = ['PHONE_NUMBER', 'ADDRESS', 'LOCATION', 'PERSON', 'EMAIL_ADDRESS'],
        language = 'en'
    )
    return anonymizer.anonymize(text=data, analyzer_results=result).text if result else data
  else: return data

file_upload = files.upload()
for filename in file_upload.keys():
  if filename.endswith('.json'):
    with open(filename, 'r') as fp:
      json_data = json.load(fp)
      processed_data = processing_json(json_data)
      print(json.dumps(processed_data, indent = 4))





Saving test.json to test.json
{
    "name": "<PERSON>",
    "age": 25,
    "city": "<PERSON>",
    "married": false,
    "hobbies": [
        "reading",
        "traveling",
        "programming"
    ],
    "address": {
        "street": "Lot 773 <PERSON>",
        "city": "Coober <LOCATION>",
        "postal_code": "5723"
    },
    "country": "<LOCATION>",
    "phone": "<PHONE_NUMBER>"
}
